In [1]:
import pandas as pd
import numpy as np

In [3]:
raw_file = '../examples/logs/sample/DiseaseReports.tsv'
raw_data = pd.read_csv(raw_file, sep = "\t|,", engine='python')
infectious_data = raw_data[raw_data['diseaseStatus'] == 'Infectious'].reset_index()
infectious_data

,index,step,agentId,[regionId,diseaseStatus,diseaseSeq,time,location,checkin,Reported,Sampled,Sequenced]
0,115,0,115,[16,Infectious,115-1,2019-07-01T00:00:00.000,POINT (740685.0388442752 3737074.5163876633),AtHome,True,False,false]
1,202,1502,116,[16,Infectious,115-1.116-1,2019-07-06T05:10:00.000,POINT (740213.7360802672 3736859.6693362864),AtHome,True,False,false]
2,203,1520,108,[15,Infectious,115-1.108-1,2019-07-06T06:40:00.000,POINT (739325.580315399 3737452.977387365),AtHome,False,False,false]
3,219,1944,119,[16,Infectious,115-1.108-1.119-1,2019-07-07T18:00:00.000,POINT (743214.0607440356 3737987.865882812),AtRecreation,True,False,false]
4,224,2211,190,[26,Infectious,115-1.108-1.190-1,2019-07-08T16:15:00.000,POINT (743694.5768428347 3738063.3984664497),AtWork,True,False,false]
...,...,...,...,...,...,...,...,...,...,...,...,...
181,724,7673,62,[8,Infectious,115-1.108-1.128-1.8-1.14-1.5-1.69-1.62-1,2019-07-27T15:25:00.000,POINT (742505.4730519045 3738614.753988403),AtHome,True,False,false]
182,733,7958,54,[8,Infectious,115-1.108-1.128-1.36-1.57-1.50-1.54-1,2019-07-28T15:10:00.000,POINT (743058.3326469811 3738742.4355007755),AtHome,True,False,false]
183,735,8073,59,[8,Infectious,115-1.108-1.128-1.181-1.182-1.105-1.122-1.59-1,2019-07-29T00:45:00.000,POINT (742992.987900245 3738738.9273155313),AtHome,False,False,false]
184,743,8634,29,[4,Infectious,115-1.108-1.128-1.8-1.14-1.5-1.69-1.62-1.29-1,2019-07-30T23:30:00.000,POINT (742327.8171828259 3739798.407336465),AtHome,True,False,false]


### TODO Notes:
- Create a tree while the simulation running
- Transform the tree into a tree with all cases on the node

In [4]:
import os
if not os.path.exists('sample-output'):
    os.makedirs('sample-output')

In [6]:
raw_tree_file = open('sample-output/tree.txt', 'w')

In [49]:
raw_tree_file.close()

In [38]:
seq_list = list(infectious_data['diseaseSeq'])
time_list = list(infectious_data['step'])

#seq_list = ['1','1.2','1.3','1.2.4','1.2.5','1.3.6','1.3.6.7']
#time_list = [0, 3, 4, 5, 6, 9, 10]

seq_time_dict = {}
for i in range(len(seq_list)):
    seq_time_dict[seq_list[i]] = time_list[i]

In [50]:
forests = {}
zero_cases = []
taxa = []
for line in seq_list:
    line = line.strip()
    nodes = line.split('.')
    taxa.append(nodes[-1])
    if len(nodes) == 1:
        node = nodes[0]
        zero_cases.append(node)
        forests[node] = {node:0}
        continue
    tree_parent = forests
    ancestor = ''
    for i in range(1,len(nodes)):
        ancestor = '.'.join(nodes[:i])
        for key in tree_parent.keys():
            if ancestor in key:
                tree_parent = tree_parent[key]
                break
    time_between = ((seq_time_dict[line] - seq_time_dict[ancestor])/288 - 6)/11.0 + 0.5
    current_key = '%s:%.3f'%(line,time_between)
    tree_parent[current_key] = {nodes[-1]:0}
    

In [54]:
def dict_to_newick(d, parent_length=0):
    if not d:
        return ""
    
    newick = ""
    for k, v in d.items():
        if isinstance(v, dict):
            child_newick = dict_to_newick(v)
            if child_newick:
                newick += f"({child_newick}):{k.split(':')[1]}"
            else:
                newick += f"{k.split(':')[0]}:{k.split(':')[1]}"
        else:
            newick += f"{k.split(':')[0]}:{v}"
        newick += ","
    
    return newick.rstrip(',')

def write_to_newick(filename,forest, zero_patients, taxa):
    fout = open(filename,'w')
    fout.write('#NEXUS\nbegin taxa;\n\tdimensions ntax=%d;\n\ttaxlabels\n\t'%len(taxa))
    fout.write('\n\t'.join(taxa))
    fout.write('\n;\nend;\n\nbegin trees;\n')
    
    for i,zp in enumerate(zero_patients):
        fout.write('\ttree tree_%d = [&U] ('%i)
        fout.write(dict_to_newick(forest[zp]))
        fout.write(');\nend;\n')
    
    fout.close()

In [55]:
write_to_newick('sample-output/tree.txt', forests, zero_cases, taxa)